In [2]:
print("LENET START")

LENET START


In [5]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.autograd import Variable
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

learning_rate = 1e-3
batch_size = 10
epoches = 50

trans_img = transforms.Compose([
        transforms.ToTensor()
    ])

trainset = MNIST('./data', train=True, transform=trans_img,download = True)
testset = MNIST('./data', train=False, transform=trans_img,download = True)

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

# build network
class Lenet(nn.Module):
    def __init__(self):
        super(Lenet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 3, stride=1, padding=1),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5, stride=1, padding=0),
            nn.MaxPool2d(2, 2)
        )

        self.fc = nn.Sequential(
            nn.Linear(400, 120),
            nn.Linear(120, 84),
            nn.Linear(84, 10),
            nn.LogSoftmax(dim = 0)
        )


    def forward(self, x):
        out = self.conv(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

lenet = Lenet()
lenet.cuda()

criterian = nn.NLLLoss(size_average=False)
optimizer = optim.SGD(lenet.parameters(), lr=learning_rate)

# train
for i in range(epoches):
    running_loss = 0.
    running_acc = 0.
    for (img, label) in trainloader:
        img = Variable(img).cuda()
        label = Variable(label).cuda()

        optimizer.zero_grad()
        output = lenet(img)
        loss = criterian(output, label)
#         print output
        # backward
        loss.backward()
        optimizer.step()

        running_loss += loss.data[0]
        _, predict = torch.max(output, 1)
        correct_num = (predict == label).sum()
        running_acc += correct_num.data[0]

    running_loss /= len(trainset)
    running_acc /= len(trainset)

    print("[%d/%d] Loss: %.5f, Acc: %.2f" %(i+1, epoches, running_loss, 100*running_acc))

[1/50] Loss: 0.97366, Acc: 74.94
[2/50] Loss: 0.72124, Acc: 82.42
[3/50] Loss: 0.69217, Acc: 83.21
[4/50] Loss: 0.67871, Acc: 83.69
[5/50] Loss: 0.67605, Acc: 83.82
[6/50] Loss: 0.66503, Acc: 84.00
[7/50] Loss: 0.65662, Acc: 84.45


Process Process-75:
Process Process-74:
Process Process-73:
Process Process-76:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ryan/anaconda2/envs/deeplearning/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/home/ryan/anaconda2/envs/deeplearning/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/home/ryan/anaconda2/envs/deeplearning/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/home/ryan/anaconda2/envs/deeplearning/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
    self.run()
    self.run()
  File "/home/ryan/anaconda2/envs/deeplearning/lib/python2.7/multiprocessing/process.py", line 114, in run
    self.run()
  File "/home/ryan/anaconda2/envs/deeplearning/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/ryan/anaconda2/envs/deeplearning/lib

KeyboardInterrupt: 

KeyboardInterrupt
    buf = self.recv_bytes()
  File "/home/ryan/anaconda2/envs/deeplearning/lib/python2.7/site-packages/torchvision/transforms/transforms.py", line 42, in __call__
KeyboardInterrupt
    img = t(img)
  File "/home/ryan/anaconda2/envs/deeplearning/lib/python2.7/site-packages/torchvision/transforms/transforms.py", line 61, in __call__
    return F.to_tensor(pic)
  File "/home/ryan/anaconda2/envs/deeplearning/lib/python2.7/site-packages/torchvision/transforms/functional.py", line 76, in to_tensor
    return img.float().div(255)
  File "/home/ryan/anaconda2/envs/deeplearning/lib/python2.7/site-packages/torch/tensor.py", line 43, in float
    return self.type(type(self).__module__ + '.FloatTensor')
  File "/home/ryan/anaconda2/envs/deeplearning/lib/python2.7/site-packages/torch/_utils.py", line 38, in _type
    return new_type(self.size()).copy_(self, async)
KeyboardInterrupt


In [3]:
lenet.eval()

testloss = 0.
testacc = 0.
for (img, label) in testloader:
    img = Variable(img).cuda()
    label = Variable(label).cuda()

    output = lenet(img)
    loss = criterian(output, label)
    testloss += loss.data[0]
    _, predict = torch.max(output, 1)
    num_correct = (predict == label).sum()
    testacc += num_correct.data[0]

testloss /= len(testset)
testacc /= len(testset)
print("Test: Loss: %.5f, Acc: %.2f %%" %(testloss, 100*testacc))

Test: Loss: 0.08437, Acc: 98.40 %
